# GEORREFERENCIAMENTO OVP-DH
## Jupyter Notebook compartilhado

In [1]:
import numpy as np
import pandas as pd
import folium
from folium import plugins

### Abrindo dataset

In [2]:
data = pd.read_csv('../data/data_geo_bo2016.csv').drop(columns=['Unnamed: 0'])

In [3]:
data.head()

,NUM_BO,RUBRICA,ANO,MES,CIDADE,LOGRADOURO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE
0,943,"Furto qualificado (art. 155, §4o.)",2016,1,S.PAULO,RUA JOAO MOURA,500,-23.560901,-46.677317
1,992386,Roubo (art. 157),2016,7,S.ANDRE,RUA MAURITI,60,-23.674312,-46.504567
2,1044074,Furto (art. 155),2016,8,S.PAULO,AVENIDA PRESIDENTE WILSON,4800,-23.594249,-46.589926
3,1996,Drogas sem autorização ou em desacordo (Art.33...,2016,3,S.PAULO,RUA DARIO VELOSO,130,-23.477746,-46.746526
4,815370,Roubo (art. 157),2016,6,S.PAULO,RUA MARIO COVAS,0,-23.505120,-46.367294


In [4]:
print('hello world!')

hello world!


## Construindo o mapa 

### Definindo 'RUBRICA' como objetivo de pesquisa

In [5]:
print('Numero de rubricas: ', data['RUBRICA'].nunique(), '\n')
print('Descricao das rubricas: \n', data['RUBRICA'].unique())

Numero de rubricas:  13 

Descricao das rubricas: 
 ['Furto qualificado (art. 155, §4o.)' 'Roubo (art. 157)'
 'Furto (art. 155)'
 'Drogas sem autorização ou em desacordo (Art.33, caput)'
 'Lesão corporal (art 129 § 9º)' 'Lesão corporal (art. 129)'
 'Lesão corporal culposa na direção de veículo automotor (Art. 303)'
 'A.I.-Estupro de vulneravel (art.217-A)' 'A.I.-Roubo (art. 157)'
 'Estupro de vulneravel (art.217-A)'
 'Homicídio culposo na direção de veículo automotor (Art. 302)'
 'A.I.-Drogas sem autorização ou em desacordo (Art.33, caput)'
 'A.I.-Lesão corporal (art. 129)']


### Reformulando endereco, crime, lei, e data

In [6]:
# criando endereco
data['ENDERECO'] = data['CIDADE'].str.strip() + ', ' + data['LOGRADOURO'].str.strip()

In [7]:
# obtendo o crime constatado na rubrica
data[['CRIME','ARTIGO']] = data['RUBRICA'].str.split('(', expand=True)

In [8]:
# limpando os atributos criados
data.CRIME = data.CRIME.str.strip()
data.ARTIGO = data.ARTIGO.str.strip(')')

In [9]:
# observando a coluna de crimes
data.CRIME.unique()

array(['Furto qualificado', 'Roubo', 'Furto',
       'Drogas sem autorização ou em desacordo', 'Lesão corporal',
       'Lesão corporal culposa na direção de veículo automotor',
       'A.I.-Estupro de vulneravel', 'A.I.-Roubo',
       'Estupro de vulneravel',
       'Homicídio culposo na direção de veículo automotor',
       'A.I.-Drogas sem autorização ou em desacordo',
       'A.I.-Lesão corporal'], dtype=object)

In [10]:
# observando a coluna de leis
# a formatacao nao esta perfeita, mas nao ha artigos iguais com diferentes referencias
data.ARTIGO.unique()

array(['art. 155, §4o.', 'art. 157', 'art. 155', 'Art.33, caput',
       'art 129 § 9º', 'art. 129', 'Art. 303', 'art.217-A', 'Art. 302'],
      dtype=object)

In [11]:
# funcao para declarar os artigos homogeneamente
def PretifyArtigo(artigo):
    artigo = artigo.capitalize().strip().strip('.')
    if "." in artigo:
        if ". " in artigo:
            return artigo
        else:
            artigo = lambda x
            
# se tiver um um ponto:
# se tiver espaco depois do ponto:
# se nao tiver, coloque

SyntaxError: invalid syntax (<ipython-input-11-2c6be5816963>, line 2)

In [36]:
# aplicando a funcao a coluna de artigos



In [ ]:
# criando coluna de datetime
data["DATA"] = data.ANO.astype(str) + "-" + data.MES.astype(str)

data["DATA"] = pd.to_datetime(data["date"], format="%Y-%m", )

In [74]:
# Pegando a média da latitudes e longitudes das ocorrências
latmean = data['latitude'].mean()
lonmean = data['longitude'].mean()

# Criando o mapa
mapa_popup = folium.Map(location=[latmean,lonmean], 
                         zoom_start=8, 
                         tiles=None, 
                         control_scale=True)

# Determinando as texturas do mapa
texturas = [('openstreetmap', 'textura: tradicional'),
            ('stamenterrain', 'textura: terreno'),
            ('stamenwatercolor','textura: desenho')]

for tile, name in texturas:
    folium.TileLayer(tiles=tile, name=name).add_to(mapa_popup)

# Minimapa
minimap = plugins.MiniMap(toggle_display=True)
mapa_popup.add_child(minimap)

# Plugins - fullscreen
plugins.Fullscreen(position='topleft').add_to(mapa_popup)

### Definindo as cores

In [75]:
dic_nl = {'Crimes de abuso de autoridade (Lei 4.898/1965)':'pink',
          'Crimes de lesão corporal (CP)':'darkred',
          'Crimes contra a honra (CP)':'green',
          'Crimes contra a liberdade pessoal (CP)':'white',
          'Crimes contra a liberdade sexual (CP)':'purple',
          'Crimes contra a vida (CP)':'red',
          'Crimes contra a administração da justiça (CP)':'cadetblue',
          'Crimes de tortura (Lei 9.455/1997)':'black',
          'Crimes contra a administração em geral (CP)':'lightgreen',
          'Crimes contra a paz pública (CP)':'darkgreen',
          'Crimes de periclitação da vida e da saúde (CP)':'darkpurple',
          'Crimes de dano (CP)':'lightred',
          'Crimes de perigo comum (CP)':'orange',
          'Abuso de autoridade do ECA (Lei 8.069/1990)':'darkblue',
          'Crime de extorsão (CP)':'gray',
          'Crimes contra a fé pública (CP)':'blue',
          'Convenção contra o crime de genocídio':'beige'}

### Plotando sub-grupos em camadas

In [76]:
all_subgroups = folium.FeatureGroup(name='Tudo')

mapa_popup.add_child(all_subgroups);

In [77]:
html_temp = """
    <h5> Natureza da lesao: </h5>
    {}
    <h5>Conduta: </h5>
    {}
    <h5>Logradouro sede: </h5>
    {}
    """

def grupo_mapa(natureza_lesao):
    group = plugins.FeatureGroupSubGroup(all_subgroups, natureza_lesao)
    mapa_popup.add_child(group)
    
    for i in data[data['natureza_lesao'] == natureza_lesao].itertuples():
        lesao = i.natureza_lesao
        if type(lesao) != str:
            lesao = "Indefinida"
        conduta = i.conduta
        if type(conduta) != str:
            conduta = "Indefinida"
        logradouro = i.logradouro_sede 
        if type(logradouro) != str:
            logradouro = "Indefinido"
        variaveis = (lesao, conduta, logradouro)
        html = html_temp.format(lesao, conduta, logradouro)
        popup = folium.Popup(html, max_width=280, min_width=250)
        folium.Marker(location=[i.latitude, i.longitude], 
            popup=popup, 
            icon=folium.Icon(color=dic_nl[natureza_lesao], 
                           icon='bolt', 
                           prefix='fa')).add_to(group)

In [78]:
for i in dic_nl.keys():
    grupo_mapa(i)

### Adicionando menu

In [79]:
folium.LayerControl(position='bottomright').add_to(mapa_popup);

### Visualizando o mapa

In [80]:
mapa_popup

### Salvando o mapa

In [13]:
mapa_popup.save('../examples/mapa_popup.html')